In [1]:

import os
import pandapower as pp
import polars as pl
import pyomo.environ as pyo
from pyomo.environ import Suffix, SolverFactory

# Remove or adjust imports for optional plotting if not available
# from data_display.grid_plotting import plot_grid_from_pandapower
from data_connector import pandapower_to_dig_a_plan_schema

# Import the D-W pipeline
from dig_a_plan_pipline_D_W import DigAPlan


# in your original code
os.chdir(os.getcwd().replace("/src", ""))

# Ensure Gurobi license is found
os.environ['GRB_LICENSE_FILE'] = os.path.expanduser('~/gurobi_license/gurobi.lic')

In [2]:

# Load and preprocess a sample pandapower network
LOAD_FACTOR = 1.0
net = pp.from_pickle('.cache/input_data/mv_example.p')
net.load['p_mw'] *= LOAD_FACTOR
net.load['q_mvar'] *= LOAD_FACTOR
# Example adjustments (optional)
net.trafo['vn_hv_kv'] = [105.0, 110.0]
net.trafo['vn_lv_kv'] = [20.0, 21.0]
net.line.loc[16, 'max_i_ka'] = 1e-3

In [3]:
# Convert pandapower network to pipeline schema
# Should return a dict with 'node_data' and 'edge_data' as Polars DataFrames
grid_data = pandapower_to_dig_a_plan_schema(net)

# Initialize and configure the Dantzig–Wolfe pipeline
dig_dw = DigAPlan(
    solver_name='gurobi',
    big_m=1e4,
    v_penalty_cost=1e+3,
    i_penalty_cost=1e+3,
    verbose=True
)

dig_dw.add_grid_data(**grid_data)

2025-05-20 16:57:17 B01BPC06 pyomo.core[1171444] WARNING Implicitly replacing the Component attribute dual (type=<class 'pyomo.core.base.suffix.Suffix'>) on block unknown with a new Component (type=<class 'pyomo.core.base.suffix.Suffix'>).
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


=== grid_data keys === ['N', 'L', 'S', 'C', 'r', 'x', 'b', 'n_transfo', 'p_node', 'q_node', 'i_max', 'v_min', 'v_max', 'slack_node', 'slack_node_v_sq', 'big_m', 'v_penalty_cost', 'i_penalty_cost']
N (nodes): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
L (edges): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
C (first 2): {0: [(1, 15)], 1: [(2, 16)]}


In [4]:
columns = dig_dw.solve_column_generation(max_iters=10)


==== Initial Tree Pattern Debug ====
Slack node: 0
Nodes with demand:
2025-05-20 16:57:17 B01BPC06 gurobipy[1171444] INFO Read LP format model from file /tmp/tmpiyy24zjn.pyomo.lp
2025-05-20 16:57:17 B01BPC06 gurobipy[1171444] INFO Reading time = 0.00 seconds
2025-05-20 16:57:17 B01BPC06 gurobipy[1171444] INFO x1: 1 rows, 1 columns, 1 nonzeros
2025-05-20 16:57:17 B01BPC06 gurobipy[1171444] INFO Set parameter QCPDual to value 1
2025-05-20 16:57:17 B01BPC06 gurobipy[1171444] INFO Set parameter NonConvex to value 2
2025-05-20 16:57:17 B01BPC06 gurobipy[1171444] INFO Set parameter QCPDual to value 1
Read LP format model from file /tmp/tmpiyy24zjn.pyomo.lp
Reading time = 0.00 seconds
x1: 1 rows, 1 columns, 1 nonzeros
Set parameter QCPDual to value 1
Set parameter NonConvex to value 2
Set parameter QCPDual to value 1
2025-05-20 16:57:17 B01BPC06 gurobipy[1171444] INFO Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.1 LTS")
2025-05-20 16:57:17 B01BPC06 gurobipy[11714

In [5]:
# Run column-generation
try:
    columns = dig_dw.solve_column_generation(max_iters=10)
    print(f"Generated {len(columns)} columns.")
    for idx, col in enumerate(columns):
        print(f"Column {idx}: cost = {col['cost']:.6f}")
        on_branches = [edge for edge, v in col["d"].items() if v==1]
        print("  active arcs:", on_branches)
    # Final pattern
    d_last = columns[-1]['d']
    print("Final topology pattern (branches ON):")
    for (l, i, j), val in d_last.items():
        if val == 1:
            print(f"  Branch {(l,i,j)} is ON")
except Exception as e:
    print(f"Error during DW solve: {e}")


==== Initial Tree Pattern Debug ====
Slack node: 0
Nodes with demand:
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO Read LP format model from file /tmp/tmpp6ow9f_d.pyomo.lp
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO Reading time = 0.00 seconds
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO x1: 1 rows, 1 columns, 1 nonzeros
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO Set parameter QCPDual to value 1
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO Set parameter NonConvex to value 2
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO Set parameter QCPDual to value 1
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.1 LTS")
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO 
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO CPU model: Intel(R) Core(TM) i7-4790 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
2025-05-20 16:57:18 B01BPC06 gurobipy[1171444] INFO Thread count: 4 physic